# LangChain 实战：冰箱销售聊天机器人

## 使用 GPT-4 构造销售话术数据

ChatGPT 分享链接：https://chat.openai.com/share/f3e4b9b0-95fb-4c6a-a3c7-f901dd194c91


使用 ChatGPT 构造销售数据的 Prompt 示例：

```
你是中国顶级的冰箱销售，现在培训职场新人，请给出100条实用的销售话术。

每条销售话术以如下格式给出：
[客户问题]
[销售回答]

```

GPT-4 回复结果：

```
1.
[客户问题] 这台冰箱B清洁方便吗？
[销售回答] 这台冰箱B这台4642内胆采用了不粘涂层设计，清洁非常方便。

2.
[客户问题] 这台冰箱D有多少升容量？
[销售回答] 这台冰箱D的总容量是2597升，非常适合家庭使用。

3.
[客户问题] 这台冰箱A有变频功能吗？
[销售回答] 这台冰箱A是的，这台4769配备了变频技术，能够根据需求调整运行状态，更加节能。

4.
[客户问题] 这台冰箱B能效等级是什么？
[销售回答] 这台冰箱B的能效等级是4048，属于高效节能产品。

5.
[客户问题] 这台冰箱C适合小厨房吗？
[销售回答] 这台冰箱C是的，这台2358设计紧凑，非常适合小厨房使用。

6.
[客户问题] 这台冰箱D有独立冷藏室吗？
[销售回答] 这台冰箱D是的，这台1723配备了独立冷藏室，方便分类存储食物。

7.
[客户问题] 这台冰箱A有多少升容量？
[销售回答] 这台冰箱A的总容量是2256升，非常适合家庭使用。

8.
[客户问题] 这台冰箱D有独立冷藏室吗？
[销售回答] 这台冰箱D是的，这台4935配备了独立冷藏室，方便分类存储食物。

9.
[客户问题] 这台冰箱C有快速解冻功能吗？
[销售回答] 这台冰箱C是的，这台2913配备了快速解冻功能，非常实用。

10.
[客户问题] 这台冰箱D适合小厨房吗？
[销售回答] 这台冰箱D是的，这台2479设计紧凑，非常适合小厨房使用。

11.
[客户问题] 这台冰箱D有儿童锁功能吗？
[销售回答] 这台冰箱D是的，这台3862配备了儿童锁功能，使用更加安全。

12.
[客户问题] 这台冰箱A能快速制冷吗？
[销售回答] 这台冰箱A是的，这台4662采用了快速制冷技术，可以在短时间内迅速降温。

13.
[客户问题] 这台冰箱A有智能控制功能吗？
[销售回答] 这台冰箱A是的，这台1468配备了智能控制系统，可以通过手机APP进行远程操控。

14.
[客户问题] 这台冰箱C保修期是多久？
[销售回答] 这台冰箱C提供2724整机保修和10压缩机保修。

15.
[客户问题] 这台冰箱D适合小厨房吗？
[销售回答] 这台冰箱D是的，这台2186设计紧凑，非常适合小厨房使用。

```


## 使用 Document Transformers 模块来处理原始数据


将 ChatGPT 生成的结果保存到 [refrigerator_sales_dialogues_unique.txt](real_estate_sales_data.txt) 文件中

In [1]:
with open("/Users/joyce/Desktop/AI应该/refrigerator_sales_dialogues_unique.txt") as f:
    electronic_sales = f.read()

### 使用 CharacterTextSplitter 来进行文本分割

- 基于单字符来进行文本分割（separator）
- 基于字符数来决定文本块长度（chunk_size）

参考示例：

```python
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
```


In [2]:
from langchain.text_splitter import CharacterTextSplitter

In [3]:
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 100,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [4]:
docs = text_splitter.create_documents([electronic_sales])

In [5]:
docs[0]

Document(page_content='[客户问题] 这台冰箱B清洁方便吗？\n[销售回答] 这台冰箱B这台4642内胆采用了不粘涂层设计，清洁非常方便。')

In [6]:
len(docs)

50

### 使用 Faiss 作为向量数据库，持久化存储房产销售 问答对（QA-Pair）

In [7]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

#openai_api_key = '# Your OpenAI API key'

# Create the OpenAIEmbeddings instance with the API key

db = FAISS.from_documents(docs, OpenAIEmbeddings(openai_api_key=openai_api_key))

/Users/joyce/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [15]:
query = "这台冰箱A噪声大吗？"

In [16]:
answer_list = db.similarity_search(query)

In [17]:
for ans in answer_list:
    print(ans.page_content + "\n")

[客户问题] 这台冰箱A噪音大吗？
[销售回答] 这台冰箱A的噪音低于1781分贝，不会影响您的休息。

[客户问题] 这台冰箱B噪音大吗？
[销售回答] 这台冰箱B的噪音低于4190分贝，不会影响您的休息。

[客户问题] 这台冰箱A有变频功能吗？
[销售回答] 这台冰箱A是的，这台4769配备了变频技术，能够根据需求调整运行状态，更加节能。

[客户问题] 这台冰箱A适合小厨房吗？
[销售回答] 这台冰箱A是的，这台2817设计紧凑，非常适合小厨房使用。



In [18]:
db.save_local("/Users/joyce/Desktop/AI应该/sale")

### 使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [19]:
# 实例化一个 TopK Retriever
topK_retriever = db.as_retriever(search_kwargs={"k": 3})

In [20]:
topK_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x10a1f5bd0>, search_kwargs={'k': 3})

In [21]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 这台冰箱A噪音大吗？
[销售回答] 这台冰箱A的噪音低于1781分贝，不会影响您的休息。

[客户问题] 这台冰箱B噪音大吗？
[销售回答] 这台冰箱B的噪音低于4190分贝，不会影响您的休息。

[客户问题] 这台冰箱A有变频功能吗？
[销售回答] 这台冰箱A是的，这台4769配备了变频技术，能够根据需求调整运行状态，更加节能。



In [26]:
docs = topK_retriever.get_relevant_documents("这台冰箱B清洁方便吗？")

In [27]:
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 这台冰箱B清洁方便吗？
[销售回答] 这台冰箱B这台4642内胆采用了不粘涂层设计，清洁非常方便。

[客户问题] 这台冰箱E清洁方便吗？
[销售回答] 这台冰箱E这台3627内胆采用了不粘涂层设计，清洁非常方便。

[客户问题] 这台冰箱E清洁方便吗？
[销售回答] 这台冰箱E这台2447内胆采用了不粘涂层设计，清洁非常方便。



#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [28]:
# 实例化一个 similarity_score_threshold Retriever
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8}
)

In [29]:
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 这台冰箱A噪音大吗？
[销售回答] 这台冰箱A的噪音低于1781分贝，不会影响您的休息。

[客户问题] 这台冰箱B噪音大吗？
[销售回答] 这台冰箱B的噪音低于4190分贝，不会影响您的休息。

[客户问题] 这台冰箱A有变频功能吗？
[销售回答] 这台冰箱A是的，这台4769配备了变频技术，能够根据需求调整运行状态，更加节能。

[客户问题] 这台冰箱A适合小厨房吗？
[销售回答] 这台冰箱A是的，这台2817设计紧凑，非常适合小厨房使用。



### 提取向量数据库中的`销售回答`

In [30]:
docs = retriever.get_relevant_documents(query)

In [31]:
docs[0].page_content

'[客户问题] 这台冰箱A噪音大吗？\n[销售回答] 这台冰箱A的噪音低于1781分贝，不会影响您的休息。'

In [32]:
docs[0].page_content.split("[销售回答] ")

['[客户问题] 这台冰箱A噪音大吗？\n', '这台冰箱A的噪音低于1781分贝，不会影响您的休息。']

In [33]:
ans = docs[0].page_content.split("[销售回答] ")[-1]

In [34]:
ans

'这台冰箱A的噪音低于1781分贝，不会影响您的休息。'

#### 尝试各种问题

In [35]:
from typing import List

def sales(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("[销售回答] ")[-1] for doc in docs]

    return ans_list

In [37]:
query = "冰箱C有快速解冻功能吗"

print(sales(query))

['这台冰箱C是的，这台2913配备了快速解冻功能，非常实用。', '这台冰箱B是的，这台3346配备了快速解冻功能，非常实用。', '这台冰箱D是的，这台1285配备了快速解冻功能，非常实用。', '这台冰箱B是的，这台4908配备了快速解冻功能，非常实用。']


In [39]:
print(sales(query, 0.75))

['这台冰箱C是的，这台2913配备了快速解冻功能，非常实用。', '这台冰箱B是的，这台3346配备了快速解冻功能，非常实用。', '这台冰箱D是的，这台1285配备了快速解冻功能，非常实用。', '这台冰箱B是的，这台4908配备了快速解冻功能，非常实用。']


In [42]:
query = "价格2000元左右"

print(f"score:0.8 ans: {sales(query)}\n")
print(f"score:0.75 ans: {sales(query, 0.75)}\n")
print(f"score:0.5 ans: {sales(query, 0.5)}\n")

/Users/joyce/anaconda3/lib/python3.11/site-packages/langchain_core/vectorstores/base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


score:0.8 ans: []

score:0.75 ans: []

score:0.5 ans: ['这台冰箱B的售价是3999元，现在有促销活动。', '这台冰箱C的售价是1126元，现在有促销活动。', '这台冰箱B的售价是1444元，现在有促销活动。', '这台冰箱D是的，这台1699设计紧凑，非常适合小厨房使用。']



/Users/joyce/anaconda3/lib/python3.11/site-packages/langchain_core/vectorstores/base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.75
  warnings.warn(


#### 当向量数据库中没有合适答案时，使用大语言模型能力

In [53]:
from langchain.chains import RetrievalQA
#from langchain_openai import ChatOpenAI
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0.5,openai_api_key=openai_api_key)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [68]:
qa_chain({"query": query})

/Users/joyce/anaconda3/lib/python3.11/site-packages/langchain_core/vectorstores/base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '价格2000元左右',
 'result': '对不起，您的问题不完整，我需要更多的信息才能提供一个准确的答案。您想知道什么产品的价格是2000元左右吗？',
 'source_documents': []}

In [69]:
qa_chain({"query": "你们店里最便宜的冰箱是哪款"})

/Users/joyce/anaconda3/lib/python3.11/site-packages/langchain_core/vectorstores/base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '你们店里最便宜的冰箱是哪款',
 'result': '对不起，我无法回答这个问题，因为我是一个人工智能，无法获取实时的商店冰箱价格信息。',
 'source_documents': []}

In [70]:
print(sales("你们店里最便宜的冰箱是哪款"))

[]


/Users/joyce/anaconda3/lib/python3.11/site-packages/langchain_core/vectorstores/base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


## 加载 FAISS 向量数据库已有结果

In [71]:
#from langchain_openai import OpenAIEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

#db = FAISS.load_local("sale", OpenAIEmbeddings(openai_api_key=openai_api_key))

In [72]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4", temperature=0.5,openai_api_key=openai_api_key)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [73]:
qa_chain({"query": "我想买洗衣机，你们有么"})

/Users/joyce/anaconda3/lib/python3.11/site-packages/langchain_core/vectorstores/base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '我想买洗衣机，你们有么',
 'result': '对不起，我不能提供购买洗衣机的服务。我是一个人工智能，可以提供信息和回答问题，但不能进行实物交易。'}

In [74]:
# 输出内部 Chain 的日志
qa_chain.combine_documents_chain.verbose = True

In [75]:
qa_chain({"query": "我想买洗衣机，你们有么"})

/Users/joyce/anaconda3/lib/python3.11/site-packages/langchain_core/vectorstores/base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(




> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': '我想买洗衣机，你们有么',
 'result': '对不起，作为一个人工智能，我不能出售任何产品，包括洗衣机。我只能提供信息和帮助回答问题。'}

In [76]:
# 返回向量数据库的检索结果
qa_chain.return_source_documents = True

In [77]:
result = qa_chain({"query": "我想买洗衣机，你们有么"})

/Users/joyce/anaconda3/lib/python3.11/site-packages/langchain_core/vectorstores/base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(




> Entering new StuffDocumentsChain chain...

> Finished chain.


In [78]:
result

{'query': '我想买洗衣机，你们有么',
 'result': '对不起，我不能帮您购买洗衣机。我是一个人工智能，主要用于提供信息和回答问题。',
 'source_documents': []}